In [ ]:
import streamlit as st
import os
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings , ChatNVIDIA
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.chains import create_retrieval_chain
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv()

## Load the NVidia key

os.environ["NVIDIA_API_KEY"] = os.getenv("NVIDIA_API_KEY")

llm = ChatNVIDIA(model="meta/llama-3.3-70b-instruct")

def vector_embedding():
    if "vectors" not in st.session_state:
        st.session_state.embeddings = NVIDIAEmbeddings()
        st.session_state.loader = PyPDFDirectoryLoader(search_path="./pdfs")

        st.session_state.text_splitter = RecursiveCharacterTextSplitter(chunk_size=700, chunk_overlap=50)
        st.session_state.final_documents = st.session_state.text_splitter.split_documents(st.session_state.docs[:30])
        st.session_state.vector = FAISS.from_documents(st.session_state.final_documents, st.session_state.embeddings)

st.title("RAG Q&A Conversation with pdf including chat history")
st.markdown("## RAG Q&A Conversation with pdf including chat history")

prompt = ChatPromptTemplate.from_template(
    """
    You are a helpful assistant for question answering tasks , use the following pieces of retrieval context to answer the question. If you don't know the answer, just say that you don't know , don't try to make up an answer.
    {context}
    Question: {question}
    Helpful Answer:
    """
)

prompt1 = st.text_input("Enter the prompt:" )

if st.button("Document Embedding"):
        vector_embedding()
        st.write("Embedding Completed")

if prompt1:
    document_chain = create_stuff_documents_chain(
        llm,
        prompt=prompt,
    )
    retriever = st.session_state.vector.as_retriever()
    retriver_chain = create_retrieval_chain(
        retriever=retriever,
        combine_documents_chain=document_chain,
    )
    response = retriver_chain.run(prompt1)
    st.write(response)